# Module 3: Advanced Scala
#### Written by Adam Izraelevitz (adamiz@berkeley.edu)

### Prerequesites
**Module 1: Basic Scala**

## Table of Contents
In this tutorial, you will learn some advanced features of Scala which enable you to write more powerful and expressive Chisel generators. We will focus on the following Scala features:

**[Functional Programming](#funcprog)**
1. [Immutable vs. Mutable Data Structures](#data)  
2. [A Simple Example](#example)  
2. [Map](#map)  
2. [Anonymous Functions](#anonfunc)  
2. [Filter](#filter)  
2. [Reduce](#reduce)  
2. [FoldLeft](#foldleft)

**[Objects and Classes](#objclass)**
2. [Companion Objects](#compobj)  
2. [Case Classes](#caseclass)

**[Type System](#types)**
2. [Static Types](#static)  
2. [Common Idioms](#idiom)
2. [Generic Types](#generic)

**[Match/Case Statements](#match)**

**[Implicits](#implicits)**



# Functional Programming<a name="funcprog"></a>

Functional programming enables progammers to operate on immutable datastructures in a concise and powerful way. Before we delve into this style of programming, we need to understand the difference between immutability and mutability.

## Immutable vs Mutable Datastructures<a name="data"></a>

A datastructure is immutable if, after it is created, it cannot be changed. Conversely, mutable datastructures can be changed after creation.

Like array's in C or ArrayBuffer's in Java, Scala has a corresponding mutable List-like datastructure, `ArrayBuffer[String]`. We can use various update and appending operations to change its contents.

In [ ]:
import scala.collection.mutable //Import mutable collections library
val arraybuf = mutable.ArrayBuffer[String]("a", "b", "c")
println(arraybuf)
// Because ArrayBuffer is mutable, you can update or append values to arraybuf
arraybuf(0) = "A"
println(arraybuf)
arraybuf += "d"
println(arraybuf)

Alternatively, we can use an immutable datastructure, `List[String]`. We cannot use update or appending operations, but instead must create new immutable datastructures containing our changes:

In [ ]:
// Because List is immutable, l will always have the 3 elements "a", "b", and "c"
val list = List("a", "b", "c")
println(list)
// We cannot append a new value or update a value
// list(0) = "A" <- Illegal
// list += "d" <- Illegal

// Instead, we must create a new List with our appended or updated value
val lAppended = list ++ List("d") // ++ returns a new immutable List with both input lists appended
println(lAppended)
val lUpdated = List("A") ++ list.tail // tail returns a new List without the first element
println(lUpdated)

Immutable datastructures have the benefits of eliminating side-effects, making your code's behavior more reliable and predictable.Scala style promotes using immutable datastructures whenever possible; the only exceptions are when a mutable datastructure enables more performant or significantly cleaner code.

## A Simple Example<a name="example"></a>

Generally, we use functional programming to cleanly and concisely create new immutable datastructures from other immutable datastructures.

Suppose we are given a sequence of strings, and want to return a new sequence with each string "doubled":
```scala
val sequence = List("a", "b", "c")
println(sequence.mkString(" ")) //"a b c"
...
val newSequence = ...
...
println(newSequence.mkString(" ")) //"aa bb cc"
```

Using a for-loop and a mutable list, we can mimic how one could do this in C or Java:

In [ ]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

val doubleABuf = mutable.ArrayBuffer[String]()
for(letter <- sequence) {
    doubleABuf += letter + letter
}
println(doubleABuf.mkString(" "))

However, using functional programming, we can do the same but much more concisely:

In [ ]:
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

What's going on here?? Let's break it down.

## Map<a name="map"></a>
First, let's rewrite the same example to be more less syntactically concise, but more readable to those unfamiliar with functional programming.

In [ ]:
println(sequence.mkString(" "))

def double(s: String): String = s + s
val doubleSeq = sequence.map(double)
println(doubleSeq.mkString(" "))

As you can see, the function `double` takes a `String` and returns a new "doubled" `String`. We then pass this function, as an argument, to another function called `map`. When called, map returns a new `List` containing the results of applying `double` to every element in `sequence`.

Let's try a different function, `triple`, which appends three copies of a string together:

In [ ]:
println(sequence.mkString(" "))

def triple(s: String): String = ???
val tripleSeq = sequence.map(triple)
println(newSequence.mkString(" "))

As you can see, we can easily change the function we pass the `map`, while abstracting away the actual traversing of the sequence.

## Anonymous Functions<a name="anonfunc"></a>
In our earliest example, we had the following syntax.
```scala
s => s + s
```
This is an anonymous function, or a function that has no name.Think of this like an object with a single method. When we call an anonymous function (e.g. passing it arguments), it calls its method on those arguments. In this case, our argument `s` is doubled.

The following example shows us creating an anonymous function, then calling it:

In [ ]:
val anon = {x: Int => x + x} // Scala usually requires anonymous functions to be wrapped in {}
val result = anon(10)
println(result)

You can see that this is very similar to a Scala method declaration, `def`. Generally, we use anonymous functions when they are simple and referenced only once, and use method declarations otherwise.

Now, you should be able to fully understand our functional programming example:

In [ ]:
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

Try writing your own anonymous function, so that the example returns a sequence of tripled strings:

In [ ]:
val tripleSeq = sequence.map(s => ???)
println(tripleSeq.mkString(" "))

## Filter<a name="filter"></a>

Like `map`, the function `filter` takes a function as an argument, but this function returns true or false. Thus, calling `filter` will returns a new sequence containing only elements that return true.

The following code prints a new sequence that only contains numbers greater than 5:

In [ ]:
val numbers = Seq(1, 2, 10, 6, 3, 9)
val filtered = numbers.filter(i => i > 5)
println(filtered.mkString(" "))

## Reduce<a name="reduce"></a>

The function `reduce` will iteratively call its input function on two elements, returning a new element, until only one remains.

The following code prints the total sum of all integers in the sequence:

In [ ]:
val sum = numbers.reduce((a, b) => a + b)
println(sum)

## FoldLeft<a name="foldleft"></a>

Reductions have the problem of failing when the input sequence is empty:

In [ ]:
val empty = Seq[Int]()
empty.reduce((a, b) => a + b)

In these instances, we can use the function `foldLeft`. This function takes two arguments, an initial value and another function. This function takes two arguments - one is the value you are collecting, and the second is an element of the sequence, and it must return the new value you are collecting.

As it turns out, we can use `foldLeft` to implement our `map`, `reduce`, and `filter` examples!

In [ ]:
val mapped = sequence.foldLeft(Seq[String]()){
    (seq, element) => seq ++ Seq(element + element)
}
println(mapped.mkString(" "))

val reduced = numbers.foldLeft(0){
    (sum, element) => sum + element
}
println(reduced)

val filtered = numbers.foldLeft(Seq[Int]()){
    (seq, element) => if (element > 5) seq ++ Seq(element) else seq
}
println(filtered.mkString(" "))

As you can see, `foldLeft` is a very powerful construct, and well worth starting at these examples to fully understand them.

Other useful sequence manipulators can be found here: http://www.scala-lang.org/api/current/scala/collection/immutable/List.html

# Objects and Classes<a name="objclass"></a>

## Companion Objects<a name="compobj"></a>
Like Java, Scala has classes you can define and instantiate. However, when you want a static class, there is no direct support in Java.

Scala has a language feature for these singleton classes, called objects. Often they define a method `apply`, which is  special in that it is called by reference the object, followed by an argument list.

In [ ]:
object MyObject {
    def hi: String = "Hello World!"
    def apply(msg: String) = msg
}
println(MyObject.hi)
println(MyObject(msg)) // equivalent to MyObject.apply(msg)

When a class and an object share the same name and defined in the same file, the object is called a **companion object**.

Companion objects are usually used to run code before or after the class constructor. In the example below, we will count the number of instances of Animal. Note that you must call the companion object's `apply` method for `numberOfAnimals` to be incremented.

In [ ]:
object Animal {
    var numberOfAnimals = 0
    def apply(name: String) = {
        numberOfAnimals += 1
        new Animal(name)
    }
}
class Animal(val name: String)

val bunny = new Animal("Hopper") // Calls the class Animal's constructor
val cat = Animal("Whiskers")     // Calls the object Animal's apply method
println(Animal.numberOfAnimals)  // Prints only one animal!

Chisel uses many companion objects, like Module. When you write `Module(new MyModule)`, you are calling the Module companion object, so Chisel can run background code before and after instantiating `MyModule`.

Can call private members.

Here's another example.

A companion object is similar to a class but can only occur once.  First let's assume we have the definition of some class X.  Along with the class we will define an **companion object** for X

In [ ]:
class X(x: Int, y: Int) { val z = x + y }

object X {
  val const = 77
  def apply(i: Int): X = { new X(i, const)  }
  def apply(i: Int, j: Int): X = { new X(i, j) }
  def show(x: X): Unit = { println(s"x contains value z = ${x.z}") }
}

Our companion object X does 3 things
1. It defines a constant, companion object are the canonical way to define constants related to a class
  * ```val const = 77``` 
1. It defines two **apply** methods.  In this case the **apply** methods are known as factory methods in that they return instances of the **class X**
  * ```def apply(i: Int): X = { new X(i, const)  }``` creates an instance of X using only one argument, it uses it's constant const to provide the other missing argument
  * ```def apply(i: Int, j: Int): X = { new X(i, j) }``` defines a basic factory that requires the same arguments as the constructor for **class X**
  * These factory methods can be called naively like this ```val x1 = X.apply(4)``` or ```val x2 = X.apply(33)``` which eliminates the need to use the new keyword
  * But the real magic is that the compiler assumes the apply method any time it sees parentheses applied to an instance or object.  By way of example 
    *  ```val x1 = X(4)``` is equivalent to ```val x1 = X.apply(4)```
    * ```val x2 = X(33)``` is equivalent to ```val x2 = X.apply(33)```
  * Factory methods, usually provided via companion objects, allows alternative ways to express instance creations, they can provide additional tests for constructor parameters, conversions, and eliminate the need to use the keyword new

> We will not here that there is another way of creating a default factory method for a class via the case modifier in class definition ```case class X(...) { ... }``` which implicitly creates a factory method that does not require new and does a number of useful functions.  There are restrictions on case classes, but they are a commonly used scala idiom.

## Case Classes<a name="caseclass"></a>

Chisel often uses case classes, a declaration looks something like 


In [ ]:
case class Drill(variableSpeed: Boolean, amps: Int, rpm: Int)



The case class provides a bunch of useful features, the ones most commonly taken advantage of are:
- Allows external access to the class parameters.  These are not accessible by default a class declared with ```class X(y: Int)``` would not allow the programmer to reference ```x.y``` when x is an instance of X created via ```val x = new X(88)```
- Eliminates the need to use new when instantiating the class, i.e. one can write ```val d = Drill(true, 10, 3000)```
- Automatically creates an unapply method that supplies access to all of the class Parameters.  This allows a programmer to us match on the class as we discussed in [Scala Match and Matching](Scala-Things-You-Should-Know#scala-match-and-matching). So assuming a match is being done on an instance of Drill, once can write.

# Type System<a name="types"></a>

## Static Types<a name="static"></a>

## Common Idioms<a name="idioms"></a>

### Option

### Seq/List

## Generic Types<a name="generic"></a>

# Match/Case Statements<a name="match"></a>

Scala is a strongly typed language, so the types of all objects are known during runtime. To use this type information to dictate control flow, we use **match** statements.

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
    x match {
        case s: String => println(s"$x is a String")
        case s: Int    => println(s"$x is an Int")
        case s: Double => println(s"$x is a Double")
    }
}

Match statements have some limitations. Because Scala runs on the JVM, and the JVM does not maintain polymorphic types, you cannot match on them at runtime (because they are all erased). Note that the following example always matches the first case statement, because the `[String]`, `[Int]`, and `[Double]` polymorphic types are erased, and the case statements are **actually** matching on just a `Seq`.

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
    x match {
        case s: Seq[String] => println(s"$x is a String")
        case s: Seq[Int]    => println(s"$x is an Int")
        case s: Seq[Double] => println(s"$x is a Double")
    }
}

The scala *matching* concept is used throughout chisel and needs to be part of any chisel programmers basic understanding. Scala provides the match operator which supports:
- Simple testing for alternatives, something like a *C* switch statement
- More complex testing of ad-hoc combinations of values
- Taking actions based on the type of a variable when it's type is unknown or underspecified, for example when
  - variable is taken from a heterogeneous list ```val mixedList = List(1, "string", false)```
  - or variable is known to be a member of a super-class but not which specific sub-class it is.
- Extraction of sub strings of a string that are specified with a *regular expression*

In [ ]:
// y is an integer variable defined somewhere else in the code
val y = 7
/// ...
val x = y match {
  case 0 => "zero"
  case 1 => "one"
  case 2 => "two"
  case _ => "many"
}
println("y is " + x)

The match operator checks possible values and for each case returns a string.  A couple of things to note:
- A match is searched in the order of the case statements, once a case statement has been matched, no other
checks against other case statements are made.
- The use of underscore as a wildcard, to handle any value not found. 


# Testing adhoc combinations with match
Here's a simple example of a truth table implemented with a match statement and tuple of values


In [ ]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}

# Processing a heterogenous list with match

In [ ]:
val mixedList = List(1, "string", false, 1.57)
val stringList = mixedList.map { 
  case i: Int => i.toString
  case s: String => s
  case b: Boolean => 
    if(b) {
      "T"
    } else {
      "F"
    }
  case _ =>
    "unsupported type"
}

```stringList``` will be a list of strings ```List("1", "string", "F", "unsupported type")```
What's going on?  We have created a list that contains three different types of elements.  We are going to map this string into another string.  The ```map``` method on string implicitly *match*es each element against the cases.  Other things to note:
- Each code block that follows a the ```=>``` operator continues until it reaches either the ending brace of the match or the next case statement.
- The map statement is sometimes written out more explicitly as follows, See: [Parameterized Code Blocks](Scala-Things-You-Should-Know#Parameterized-Code-Blocks)


In [ ]:
val stringList2 = mixedList.map {  element =>
  element match {                                 
      case i: Int => i.toString
      case s: String => s
      case b: Boolean => 
        if(b) {
          "T"
        } else {
          "F"
        }
      case _ =>
        "unsupported type"
  }
}

This works as it did before, but uses the parameterized code block convention to pass each member of list to the ```map``` code block as the variable ```element```.  In this code block each element is processed by the ```match``` statement.


# Unapply methods and scala match
Scala unapply methods are another form of syntactic sugar giving match statements the ability to both match on types and extract values from those types during the matching.  Scala unapply methods can be coded explicitly by developers or, more commonly, can be generated by the use of case classes.  See [Case Classes Section](Scala-Things-You-Should-Know#Case-Classes).  There are reams of web resources on this but we'll try to capture a number of important features in hopefully not too confusing example.
First let's define a number of classes

In [ ]:
trait Component { def name: String }
case class Monitor(name: String, diagonal: Double, scanRate: Int) extends Component
case class Keyboard(name: String, numberOfKeys: Int) extends Component
case class Mouse(name: String) extends Component

We have defined a trait, a lightweight super-class, and we have created a number of case classes that extend Component.  In each case they provide a name, required when extending the trait and other parameters based on their type.  We will now create a collection of components called System which will provide a displayComponents method that uses match to identify them.

In [ ]:
class System(val components: List[Component]) {
  def displayComponents(): Unit = {
    components.foreach {
      case Monitor("BrandX", size, _) =>
        // brandx is square, lists horizontal size, all are 60Hz
        val diag = size * 1.414
        println(s"monitor BrandX of diagonal size $diag scan rate is 60")
      case Monitor(name, size, scan) =>
        println(s"monitor $name of diagonal size $size scan rate $scan")
      case Keyboard(name, keys) =>
        println(s"keyboard $name with $keys keys")
      case _: Mouse =>
        println("There is a mouse")
      case x: Component =>
        println(s"unknown component ${x.name}")
    }
  }
}

The next case statement ```case Monitor(name, size, scan) =>``` will *match* any Monitor instance (except for any with BrandX because the prior case statement caught all those).  In this case when the component is a Monitor we grab the the parameters of the Monitor class as name, size and scan, and we use those directly to print out this type of monitor.


We can begin to see the power of the match capability.  ```def displayComponents(): Unit = {``` declares the displayComponents method that returns nothing.  ```components.foreach {``` uses the foreach method to iterate over each element in the list of components. The code block of the foreach matches on a number of possibilities.

```case Monitor("BrandX", size, _) =>``` This matches a component whose type is Monitor and whose first parameter has the value. Now here is the cool part, the size symbol takes *diagonal* (the second parameter of the Monitor class) to a local variable size.  The size variable is only defined until the next case statement or to the end of the block.  Note also the use of underscore, the Monitor class has third parameter which must be dealt with, so we use the underscore to stand in for the third parameter but to indicate we do not need to use it's value.  This case statement then has a small code block
```
        // brandx is square, lists horizontal size, all are 60Hz
        val diag = size * 1.414
        println(s"monitor BrandX of diagonal size $diag scan rate is 60")
```
that does a little math on the size variable and prints out information on the monitor.

Next we match on Keyboard class instances using ```case Keyboard(name, keys) =>``` as before using match's ability to instantiate local variables with values from the class constructor parameters.  

The next case statement ```case _: Mouse =>``` uses a different match idiom.  The underscore says we have match here but we don't need to reference what was matched, but we do insist that the match be on a instance of class Mouse. See [Underscore the Scala Wildcard](Scala-Things-You-Should-Know#Underscore-the-Scala-Wildcard)

The final case statement ```case x: Component =>``` illustrates matching an instance of Component and assigning that instance to a local variable x.  Since x is a Component the only thing know about it, is that it will have a method name. So we use that method, wrapped in braces because it contains more complex scala code, to print this.

Because we declared components to be a ```List[Component]``` the compiler guarantees we don't have any other classes instances in there.  Otherwise a thorough developer should add an additional match to avoid errors.  That code would look like
```scala
    case _ =>
      // do some error handling here
```

# Implicits<a name="implicits"></a>

# Named parameters
When a method is defined in scala, for example
```scala
def myMethod(count: Int, wrap: Boolean, wrapValue: Int = 24): Unit = { ... }
```
When calling the method, you will often see the parameter names along with the passed in values
```scala
myMethod(count = 10, wrap = false, wrapValue = 23)
```
For frequently called methods, the parameter ordering may be obvious but for less common methods and, in particular, boolean arguments, including the names with calls can make your code a lot more readable.  Using named parameters can allow you to re-arrange arguments, and in combination with parameters that have a default value, can make it so the caller only has to pass (by name) the specific arguments that do not use the default value.  Parameters to class definitions also used this named argument scheme (they are actually just the parameters to the constructor method for the class).

# String interpolation
There are lot of ways of building strings with dynamic content.  The simplest is simply to use the add operator.

In [ ]:
val (dogsName, destination) = ("Rufus", "Store")
println("my dog " + dogsName + " went to the " + destination)

But there are a lot of more alternative (possibly more elegant ways) to do the same thing.  The first is the **s** string interpolator.  When a string begins with **s"** any occurrences of a dollar sign followed by a scala variable, or a dollar sign followed by a code block, will have that replaced by the string value of the variable or block.  For our previous example we could make this 

In [ ]:
println(s"my dog $dogsName went to the $destination")

As an example of using a code block, let's assume we have a list and we'd like to print this out with each animal capitalized. 


In [ ]:
val aList = List("dog", "cat", "fox")
println(s"Animal list ${aList.map(s => s.capitalize).mkString(" -- ")}")


We used the ${...} to execute some scala code on that list.  map converts the list to a new list in which each word has been capitalized, then that new list is changed into a string by taking each element and joining it to the others with the string " -- " between each element.

Another less commonly used interpolator is the **f** which allows printf style formatting specifiers to be included at each interpolation point.

In [ ]:
case class Person(name: String, age: Int)

val (lineNumber, person) = (22, Person("blink", 77))
println(f"$lineNumber%6d ${person.name}%-40s ${person.age}%3d")


Can be used to make some columns that line up nicely.  Scala does have printf too if you really need it.  But be a little careful with that Chisel provides it's own version of printf specifically for debug printing inside an executing simulation.
